In [1]:
from chestnet_utils import *

In [9]:
def get_dataloaders(dataset,bs,num_workers,use_gpu):   
    
    
    train_sampler = sampler.SubsetRandomSampler(dataset.tr_idx)
    val_sampler = sampler.SubsetRandomSampler(dataset.val_idx)
    test_sampler = sampler.SubsetRandomSampler(dataset.test_idx)
    small_train_sampler = sampler.SubsetRandomSampler(dataset.sml_tr_idx)
    small_val_sampler = sampler.SubsetRandomSampler(dataset.sml_val_idx)


    samplers = {
        'train':train_sampler,
        'val': val_sampler,
        'test':test_sampler,
        'sml_tr':small_train_sampler,
        'sml_val':small_val_sampler
    }

    dataloaders = {k : DataLoader(dataset,
                            batch_size=bs,
                            sampler=v,
                            num_workers=num_workers,
                            pin_memory=use_gpu
                           ) for k,v in samplers.items()}

    
    return samplers, dataloaders

In [10]:
tfms = get_tfms(227)

transformed_dataset = XRayDataset(f'{data_folder}train.csv',f'{data_folder}',transform=tfms)

use_gpu, num_workers = get_pm_for_dl()

samplers, dataloaders = get_dataloaders(transformed_dataset,32,num_workers,use_gpu)
dataset_sizes = get_dt_szs(samplers)

In [ ]:
model_ft = models.resnet34(pretrained=True)
for i,param in enumerate(model_ft.parameters()):
    if i ==  5:
        break
    param.requires_grad = False
# model_ft.conv1.kernel_size = (3,3)
# model_ft.avgpool = nn.AdaptiveAvgPool2d(1)
model_ft.fc = nn.Sequential(nn.Dropout(p=0.5,inplace=True),nn.Linear(2048,14))
model_ft = change_to_cuda(model_ft)

In [ ]:
model_ft.cuda()
criterion = nn.CrossEntropyLoss()
optimizer_ft = optim.Adam(filter(lambda p: p.requires_grad, model_ft.parameters()), lr=1e-4,weight_decay=1)
scheduler = optim.lr_scheduler.ExponentialLR(optimizer_ft, gamma=0.5)
# scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer_ft, 'min',patience=1,verbose=True)
dropout_model,bacc,bwt = train_model(model_ft, criterion, optimizer_ft,scheduler,dataloaders,dataset_sizes,num_epochs=10,data_split=['train','sml_val'],save_model=False)

Epoch 0/9
----------
if train
train Loss: 0.0694 Acc: 0.3012
else sml_val
sml_val Loss: 0.0773 Acc: 0.2929
setting best
Epoch 1/9
----------
if train
train Loss: 0.0644 Acc: 0.3462
else sml_val
sml_val Loss: 0.0775 Acc: 0.2429
Epoch 2/9
----------
if train
train Loss: 0.0623 Acc: 0.3742
else sml_val
sml_val Loss: 0.0735 Acc: 0.3214
setting best
Epoch 3/9
----------
if train


In [32]:
pdted,lbs = predict(model_ft,'sml_val',dataloaders,dataset_sizes,criterion)
print(np.bincount(pdted))
print(np.bincount(lbs))

sml_val
sml_val Loss: 0.0914 Acc: 0.1214
[ 0  0 24  0  0 14  0 50  0  0  1 51]
[ 3  0  7 12  6  3  3 47  9 18  7 21  3  1]
